In [1]:
import json
import os
import warnings
from pprint import pprint

import dill
import pandas as pd
from implicit.nearest_neighbours import BM25Recommender, TFIDFRecommender, CosineRecommender
from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.metrics import MAP, MeanInvUserFreq, calc_metrics, Precision, Recall, Serendipity
from rectools.model_selection import TimeRangeSplitter
from rectools.models.popular import PopularModel, Popularity

from services.userknn import UserKnn

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

## LOAD DATA

In [2]:
interactions_df = pd.read_csv('../datasets/interactions.csv')
users = pd.read_csv('../datasets/users.csv')
items = pd.read_csv('../datasets/items.csv')

interactions_df.rename(columns={'last_watch_dt': Columns.Datetime,
                                'total_dur': Columns.Weight}, inplace=True)
interactions = Interactions(interactions_df)

interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


In [4]:
items.head(5)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Грандинетти, Джеральдин Чаплин, Елена Анайя, Каэтано Велозо, Леонор Уотлинг, Лола Дуэньяс, Лолес Леон, Малу Айродо, Мариола Фуэнтес, Пас Вега, Пина Бауш, Ро...",Мелодрама легендарного Педро Альмодовара «Поговори с ней» в 2003 году получила премию «Оскар» за лучший сценарий. Журналист Марко берет интервью у знаменитой женщины-тореро Лидии и вскоре влюбляе...,"Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких,..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон Манцукас, Джон Глейсер, Карл Грин, Кристен Риттер, Лэнс Реддик, Морис Комт, Патрик Кернс, Ребекка Коллинз, Роза Салазар, Росс П. Кук, Стеффи Гроут, Ти...","Уморительная современная комедия на популярную тему о том, как не надо отмечать мальчишник. Главный герой усвоил, что не надо звать на свадьбу своего друга Джейсона, из-за которого он вместо сваде...","Голые, перцы, 2014, США, друзья, свадьбы, преодоление, трудностей, расставания, отношения, дружба, риск, недоразумение, мужская, дружба, мальчишники, девичники"
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман, Дэн Риззуто, Кендес Илэйн Калтраро, Кит Джардин, Лекса Дойг, Майкл Джей Уайт, Майкл Шэнкс, Майкл Эклунд, Питер Брайант, Питер Кент, Стив Бачич, Стив ...","Профессиональный рестлер Стив Остин («Все или ничего») и темнокожий мачо Майкл Джей Уайт («Темный рыцарь») в интригующем криминальном боевике. В центре сюжета – команда спецназовцев, которая оказа...","Тактическая, сила, 2011, Канада, бандиты, гангстеры, преступления, преодоление, трудностей, убийства, убийцы, настоящие, мужчины, риск, недоразумение, силы, правопорядка, борьба, за, выживание, сп..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джеймс, Долли Уэллс, Дэвид Сибли, Камилла Юкан, Кевин Матадин, Люси Темпл, Макс Радд, Мартин Эткинсон, Мишель Финч, Питер Дин Джексон, Пол Голдсмит, Ричард...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джеймс и Долли Уэллс в драме Эндрю Хэйя. Супруги Кейт и Джефф Мерсеры готовятся к важной дате – с того дня, как они поженились, прошло целых сорок пять л...","45, лет, 2015, Великобритания, брак, жизнь, любовь, психология, семейные, проблемы, семья, отношения, конфликты, отношениях, отношения, паре, отношения, мужчины, женщины, семейный, конфликт, смерть"
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алексей Васильев, Анатолий Папанов, Андрей Данилов, Борис Зайденберг, Борис Коковкин, Виктор Карасев, Галина Беляева, Е. Рахманова, Елена Аржаник, Ирина Са...","Расчетливая чаровница из советского кинохита «Мой ласковый и нежный зверь», актриса-дебютантка Галина Беляева снова становится неопытным и робким подростком в спортивном фильме «Все решает мгнове...","Все, решает, мгновение, 1978, СССР, сильные, женщины, спорт, преодоление, трудностей, мечта, соперничество, подростки, женщины, успех, коллектив, команда, фобии, спортсмены, отношения, между, трен..."


## Задаем фолды для кросс-валидации

In [105]:
N_SPLITS = 5
TEST_SIZE = '14D'

In [106]:
# Init generator of folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [107]:
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-06-14 00:00:00', freq='14D'),
  Timestamp('2021-06-28 00:00:00', freq='14D')),
 (Timestamp('2021-06-28 00:00:00', freq='14D'),
  Timestamp('2021-07-12 00:00:00', freq='14D')),
 (Timestamp('2021-07-12 00:00:00', freq='14D'),
  Timestamp('2021-07-26 00:00:00', freq='14D')),
 (Timestamp('2021-07-26 00:00:00', freq='14D'),
  Timestamp('2021-08-09 00:00:00', freq='14D')),
 (Timestamp('2021-08-09 00:00:00', freq='14D'),
  Timestamp('2021-08-23 00:00:00', freq='14D'))]

## Задаем метрики и модели, по которым будем делать CV

In [108]:
# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    'map@10': MAP(k=10),
    'novelty@10': MeanInvUserFreq(k=10),
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "serendipity@10": Serendipity(k=10),
}

# few simple models to compare
models = {
    "cosine_userknn": CosineRecommender(),
    "tfidf_userknn": TFIDFRecommender(),
    'b25_recommender': BM25Recommender(),
}

## CV

In [109]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.df.iloc[train_ids].copy()
    df_test = interactions.df.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()

    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=50)
        userknn_model.fit(df_train)

        recos = userknn_model.predict(df_test)

        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )

        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)



==================== Fold 0
{'end': Timestamp('2021-06-28 00:00:00', freq='14D'),
 'i_split': 0,
 'start': Timestamp('2021-06-14 00:00:00', freq='14D'),
 'test': 349230,
 'test_items': 6749,
 'test_users': 110734,
 'train': 1979424,
 'train_items': 13649,
 'train_users': 439529}


  0%|          | 0/439529 [00:00<?, ?it/s]

  0%|          | 0/439529 [00:00<?, ?it/s]

  0%|          | 0/439529 [00:00<?, ?it/s]


==================== Fold 1
{'end': Timestamp('2021-07-12 00:00:00', freq='14D'),
 'i_split': 1,
 'start': Timestamp('2021-06-28 00:00:00', freq='14D'),
 'test': 396345,
 'test_items': 6969,
 'test_users': 131767,
 'train': 2582489,
 'train_items': 14107,
 'train_users': 543840}


...


==================== Fold 4
{'end': Timestamp('2021-08-23 00:00:00', freq='14D'),
 'i_split': 4,
 'start': Timestamp('2021-08-09 00:00:00', freq='14D'),
 'test': 521381,
 'test_items': 7705,
 'test_users': 151629,
 'train': 4649162,
 'train_items': 15415,
 'train_users': 850489}


  0%|          | 0/850489 [00:00<?, ?it/s]

  0%|          | 0/850489 [00:00<?, ?it/s]

  0%|          | 0/850489 [00:00<?, ?it/s]

CPU times: total: 21h 2min 29s
Wall time: 59min 6s


# Метрики качества по фолдам 

In [110]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty@10,serendipity@10
0,0,cosine_userknn,0.004647,0.019399,0.003364,7.578876,0.000045
1,0,tfidf_userknn,0.008343,0.037900,0.006931,7.572326,0.000043
2,0,b25_recommender,0.003535,0.014183,0.002539,8.807170,0.000068
3,1,cosine_userknn,0.006532,0.029571,0.005111,7.081875,0.000045
4,1,tfidf_userknn,0.010605,0.050303,0.009431,7.101410,0.000044
5,1,b25_recommender,0.004089,0.017569,0.003009,8.519770,0.000069
6,2,cosine_userknn,0.005784,0.026333,0.004787,7.182565,0.000055
7,2,tfidf_userknn,0.008576,0.039890,0.007761,7.263590,0.000057
8,2,b25_recommender,0.003629,0.014863,0.002732,8.858159,0.000091
9,3,cosine_userknn,0.005462,0.023360,0.004260,7.333215,0.000063


In [111]:
df_metrics.groupby('model').mean()[metrics.keys()]

,map@10,novelty@10,prec@10,recall@10,serendipity@10
model,,,,,
b25_recommender,0.002686,8.880755,0.003685,0.014704,0.000089
cosine_userknn,0.004248,7.324392,0.005474,0.023718,0.000056
tfidf_userknn,0.007469,7.373960,0.008678,0.039424,0.000057


## Metrics mean

tfidf best model

## Model learning

In [82]:
final_model = UserKnn(model=TFIDFRecommender(K=30))
final_model.fit(interactions.df)

  0%|          | 0/962179 [00:00<?, ?it/s]

## Load models

In [83]:
%%time

with open(f'user_knn.dill', 'wb') as f:
    dill.dump(final_model, f)

CPU times: total: 25.8 s
Wall time: 26 s


In [22]:
with open(f'user_knn.dill', 'rb') as f:
    loaded_model = dill.load(f)

loaded_model.recommend(176549)

[15469, 5518, 12448, 6737, 5482, 10688, 4273, 5695, 7453, 5600]

## Cold users

In [91]:
items["genre"] = items["genres"].str.split(",")
items[["genre", "genres"]].head(3)

,genre,genres
0,"[драмы, зарубежные, детективы, мелодрамы]","драмы, зарубежные, детективы, мелодрамы"
1,"[зарубежные, приключения, комедии]","зарубежные, приключения, комедии"
2,"[криминал, зарубежные, триллеры, боевики, комедии]","криминал, зарубежные, триллеры, боевики, комедии"


In [92]:
genre_feature = items[[Columns.Item, "genre"]].explode("genre")
genre_feature.columns = [Columns.Item, "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,item_id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [94]:
_, bins = pd.qcut(items["release_year"], 10, retbins=True)
labels = bins[:-1]

item_feat = pd.concat([genre_feature, pd.DataFrame(
    {
        Columns.Item: items[Columns.Item],
        "value": pd.cut(items["release_year"], bins=bins, labels=bins[:-1]),
        "feature": "release_year",
    }
)])
item_feat = item_feat[item_feat[Columns.Item].isin(interactions.df[Columns.Item])]
item_feat.shape

(55676, 3)

In [103]:
dataset = Dataset.construct(
    interactions_df=interactions.df.rename(
    columns={
        'track_id': Columns.Item,
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    }, 
    inplace=False) ,
    user_features_df=None,
    item_features_df=item_feat,
    cat_item_features=['genre', 'release_year']
)

In [109]:
popular_model = PopularModel(period=pd.Timedelta('7 days 00:00:00'), popularity='n_users', add_cold=True)
popular_model.fit(dataset)

In [110]:
recs = pd.DataFrame({'user_id': interactions.df['user_id'].unique()})
cold = recs[~recs['user_id'].isin(dataset.user_id_map.external_ids)]
warm = recs[recs['user_id'].isin(dataset.user_id_map.external_ids)]
cold.head()

,user_id


In [111]:
%%time

warm_recs = popular_model.recommend(warm['user_id'], dataset=dataset, k=10, filter_viewed=True).drop(['score', 'rank'], axis=1)
warm_recs = warm_recs.groupby('user_id').agg({'item_id': list}).reset_index()
warm_recs.head()

CPU times: total: 1min 21s
Wall time: 1min 21s


,user_id,item_id
0,0,"[10440, 13865, 14488, 12360, 341, 4151, 3734, 512, 7793, 11863]"
1,1,"[9728, 15297, 13865, 14488, 12192, 12360, 341, 4151, 3734, 512]"
2,2,"[9728, 10440, 15297, 13865, 14488, 12192, 12360, 341, 4151, 3734]"
3,3,"[15297, 13865, 14488, 12360, 341, 512, 7793, 11863, 7102, 7571]"
4,4,"[9728, 10440, 15297, 13865, 14488, 12192, 12360, 341, 4151, 3734]"


In [116]:
%%time


with open('popular_model.dill', 'wb') as f:
    dill.dump(popular_model, f)

CPU times: total: 0 ns
Wall time: 1.13 ms


In [71]:
final_model = UserKnn(model=BM25Recommender(K=100), N_users=50)
final_model.fit(interactions.df)

  0%|          | 0/962179 [00:00<?, ?it/s]

In [72]:
%%time

with open(f'user_knn.dill', 'wb') as f:
    dill.dump(final_model, f)

CPU times: total: 24.6 s
Wall time: 24.9 s


In [119]:
with open(f'user_knn.dill', 'rb') as f:
    loaded_model = dill.load(f)

loaded_model.recommend(176549)

[15469, 5518, 12448, 13962, 6737, 5482, 10688, 4273, 5695, 7453]

In [85]:
%%time

predictions = loaded_model.predict(interactions.df)
predictions

CPU times: total: 3min 17s
Wall time: 3min 18s


,user_id,item_id,score,rank
0,1097557,3182,6.137841,1
1,1097557,4151,4.111983,2
2,1097557,15297,3.379502,3
4,1097557,9728,3.311465,4
3,1097557,10440,3.029132,5
...,...,...,...,...
16694812,0,7829,3.915136,6
16694814,0,12192,3.466101,7
16694807,0,9728,2.683715,8
16694808,0,10440,2.390548,9


In [120]:
import dask.dataframe as dd

a = dd.from_pandas(predictions, npartitions=50)
a

,user_id,item_id,score,rank
npartitions=50,,,,
0,int64,object,object,int64
330461,...,...,...,...
...,...,...,...,...
16359630,...,...,...,...
16694816,...,...,...,...


In [133]:
%%time

dask_df = a.groupby('user_id')['item_id'].apply(list).reset_index()

CPU times: total: 15.6 ms
Wall time: 8 ms


In [135]:
import numpy as np

dask_df['item_id'] = dask_df.apply(lambda x: np.concatenate((x['item_id'], warm_recs[warm_recs['user_id'] == x['user_id']]['item_id']), axis=None), axis=1)
dask_df['item_id'] = dask_df['item_id'].apply(lambda x: np.concatenate((x, [25, 21, 32, 16, 174, 84, 93, 142, 370, 122]), axis=None))

In [136]:
dask_df['item_id'] = dask_df['item_id'].apply(lambda x: list(dict.fromkeys(x))[:10])

In [137]:
dask_df.tail()

TypeError: unhashable type: 'list'

In [138]:
dask_df.to_json("data.json")

TypeError: unhashable type: 'list'

In [130]:
list(popular_model.popularity_list[0][:10])

[32, 25, 21, 16, 57, 148, 527, 103, 174, 84]

In [ ]:


from tqdm import tqdm
import os
import json
res = {}
for _, _,files in os.walk("data.json"):
    for file in tqdm(files):
        for line in open(f"data.json/{file}", 'r').readlines():
            i = json.loads(line)
            res[i['user_id']] = i['item_id']

res

In [ ]:
with open('final.json', 'w') as f:
    json.dump(res, f)